In [1]:
import pandas as pd

playlist_data = pd.read_csv('./spotify_data/playlistCSV.csv', header=0)
streaming_history = pd.read_csv('./spotify_data/streamingHistoryCSV.csv', header=0)
playlist_data.head()

,albumName,artistName,trackName,trackUri,playlists.name
0,...Like Clockwork,Queens of the Stone Age,I Sat by the Ocean,spotify:track:1D3YGmST01Ba0WRiT7THPG,The Knew
1,¡DOS!,Green Day,Stray Heart,spotify:track:2Q22sa3Y4lKXHHBBAxEhYY,Missing Her
2,¡DOS!,Green Day,Stray Heart,spotify:track:2Q22sa3Y4lKXHHBBAxEhYY,Perfect Playlist Pt 2
3,¡UNO!,Green Day,Nuclear Family,spotify:track:206n37mZTmloL45RYhmMlP,Missing Her
4,¡UNO!,Green Day,Nuclear Family,spotify:track:206n37mZTmloL45RYhmMlP,Rock Paper Scissors


In [2]:
merged_dataset = streaming_history.merge(playlist_data, on='trackName',)
merged_dataset['time'] = merged_dataset['time'].str.split(' ').str[1]
merged_dataset['track_id'] = merged_dataset['trackUri'].str.split('track:').str[1]


In [3]:
def month_to_int(value):
    if value == 'June':
        return 6
    elif value == 'July':
        return 7
    elif value == 'August':
        return 8
    else:
        return 9

In [4]:
merged_dataset['Month'] = merged_dataset['Month'].apply(month_to_int)

In [5]:
merged_dataset['date'] = merged_dataset['Month'].astype(str) + '-' + merged_dataset['Day'].astype(str) + '-' + merged_dataset['Year'].astype(str)
date_format = '%m-%d-%Y'
merged_dataset['date'] = pd.to_datetime(merged_dataset['date'], format=date_format)
merged_dataset['hour'] = merged_dataset['time'].str.split(':').str[0]
#merged_dataset.to_csv('./cleaned_data/mergedSpotifyData.csv', index=False)

In [6]:
merged_dataset = merged_dataset.drop_duplicates(subset='time')
merged_dataset.head()

,artistName_x,Year,Month,Day,time,Sum of msPlayed,trackName,albumName,artistName_y,trackUri,playlists.name,track_id,date,hour
0,"""Weird Al"" Yankovic",2023,6,22,02:00:00,202920,"Amish Paradise (Parody of ""Gangsta's Paradise""...",Bad Hair Day,"""Weird Al"" Yankovic",spotify:track:5r96TaQquRrlo3Ym3ZlSL2,HOmeWorK BANgers,5r96TaQquRrlo3Ym3ZlSL2,2023-06-22,02
1,"""Weird Al"" Yankovic",2023,7,29,02:38:00,202920,"Amish Paradise (Parody of ""Gangsta's Paradise""...",Bad Hair Day,"""Weird Al"" Yankovic",spotify:track:5r96TaQquRrlo3Ym3ZlSL2,HOmeWorK BANgers,5r96TaQquRrlo3Ym3ZlSL2,2023-07-29,02
2,"""Weird Al"" Yankovic",2023,7,29,01:10:00,328773,"The Saga Begins (Lyrical Adaption of ""American...",Running With Scissors,"""Weird Al"" Yankovic",spotify:track:74sUbOF9Zm8LdGUJjxleTl,HOmeWorK BANgers,74sUbOF9Zm8LdGUJjxleTl,2023-07-29,01
3,...And You Will Know Us by the Trail of Dead,2023,7,23,01:33:00,325600,Wasted State Of Mind,So Divided,...And You Will Know Us by the Trail of Dead,spotify:track:16XbElFPv3A7TrzRJntVYd,Missing Her,16XbElFPv3A7TrzRJntVYd,2023-07-23,01
5,...And You Will Know Us by the Trail of Dead,2023,8,21,03:10:00,25550,Wasted State Of Mind,So Divided,...And You Will Know Us by the Trail of Dead,spotify:track:16XbElFPv3A7TrzRJntVYd,Missing Her,16XbElFPv3A7TrzRJntVYd,2023-08-21,03


In [7]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '3811b236a5dd410e9f8bb2321bbdef9f'
client_secret = '5daa963dee6d41dd820c8d2a776c1fcc'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [8]:
import time
from tqdm import tqdm

def track_info_progress_bar(merged_dataset,api_dataset, start_value, end_value):
    requests_per_hour = 500
    
    requests_made_hour = 0
    start_time = time.time()
    
    

    for track_uri in tqdm(enumerate(merged_dataset['trackUri'][start_value:end_value]), total=end_value-start_value):
        audio_features = sp.audio_features(track_uri[1])
        new_row = pd.DataFrame(audio_features)
        api_dataset = pd.concat([api_dataset,new_row], ignore_index=True)
        time.sleep(60*3)
        
        requests_made_hour += 1
        elapsed_time = time.time() - start_time

       
        if requests_made_hour >= requests_per_hour and elapsed_time < 3600:
            time_to_sleep = 3600 - elapsed_time
            time.sleep(time_to_sleep)
            requests_made_hour = 0
            start_time = time.time()

        #tqdm.write(f"Processed: {index + 1}/{end_value}")
    return api_dataset

In [9]:
start_value = 0
end_value = 1000
api_dataset = pd.DataFrame()
api_dataset = track_info_progress_bar(merged_dataset,api_dataset, start_value, end_value)
display(api_dataset)

  0%|          | 0/1000 [00:00<?, ?it/s]

 24%|██▎       | 235/1000 [11:45:29<38:16:34, 180.12s/it]


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
start_value = 1001
end_value = 2000
api_dataset = track_info_progress_bar(merged_dataset,api_dataset, start_value, end_value, sp)
display(api_dataset)

In [ ]:
start_value = 2001
end_value = 3000
api_dataset = track_info_progress_bar(merged_dataset,api_dataset, start_value, end_value, sp)
display(api_dataset)

In [ ]:
start_value = 3001
end_value = 4000
api_dataset = track_info_progress_bar(merged_dataset,api_dataset, start_value, end_value, sp)
display(api_dataset)

In [ ]:
start_value = 4001
end_value = 5000
api_dataset = track_info_progress_bar(merged_dataset,api_dataset, start_value, end_value, sp)
display(api_dataset)

In [ ]:
columns_to_drop = ['analysis_url','track_href','type','uri']
api_dataset.drop(columns_to_drop, axis=1,inplace=True)
api_dataset.rename(columns={'id': 'track_id'}, inplace=True)


In [ ]:
spotify_dataset = merged_dataset.merge(api_dataset, on='track_id')
columns_to_drop = ['trackUri','Sum of msPlayed','artistName_x']
spotify_dataset.drop(columns_to_drop, axis=1,inplace=True)
spotify_dataset.rename(columns={'artistName_y': 'artistName'}, inplace=True)
spotify_dataset.to_csv('apiData.csv', index=False)